In [ ]:
import pandas as pd
import numpy as np
import os
import re

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")

input_file = "/home/tim/cluster/openmp-usage-analysis/result.csv"
input_file_pragmas = "/home/tim/cluster/openmp-usage-analysis/pragma_result.csv"
input_file_metadata = "/home/tim/scratch/openmp-usage-analysis-binaries/result.csv"

plot_path = "/home/tim/paper/openmp-analysis-paper/media"
df = pd.read_csv(input_file, index_col=0)
df_pragmas = pd.read_csv(input_file_pragmas, index_col=0)
df_repo_metadata = pd.read_csv(input_file_metadata, index_col=0)

In [ ]:
#pre-processing
df_repo_metadata["Code"] = df_repo_metadata["Code"].str.replace("/","--") 

In [ ]:
OVERHEAD_THRESHOLD = 100

In [ ]:
df["large_overhead"] = df["instructions_weighted"] < OVERHEAD_THRESHOLD
df["small_overhead"] = df["instructions_weighted"] >= OVERHEAD_THRESHOLD

In [ ]:
len(df["Code"].unique())

In [ ]:
pass

In [ ]:
# Number of parallel regions per Repo
ax = df.groupby("Code").size().value_counts().sort_index().plot.bar()
ax.set_xlabel("Number of parallel Regions")
ax.set_ylabel("number of Codes")
ax.set_title("Number of parallel regions per Repo")
plt.savefig(os.path.join(plot_path, "num_regions.pdf"))
plt.tight_layout()
plt.show()


In [ ]:
df["default_tripcount_loops"].sum()

In [ ]:
df["known_tripcount_loops"].sum()

In [ ]:
df["thread_dependant_trip_count_loops"].sum()

Fragen:
Wie viele user beachten die aspekte um gute openmp usage zu haben


Analyse per project

Verhältnis overhead/parallel region per project
overhead mit 100 instructions annehmen
codegröße insgesamt
Wie viele regions per project, welcher typ

welche konstrukte

In [ ]:
ax = df["instructions_flat"].plot.hist(bins=np.logspace(start=0, stop=6))
ax.set_xscale("log")
ax.set_xlabel("Instructions in Parallel region")
ax.set_title("Distribution of Instruction counts of Parallel region")
plt.savefig(os.path.join(plot_path, "flat_instructions_distribution.pdf"))

In [ ]:
# mit anderem default tripcount 10k oder so
ax = df["instructions_weighted"].plot.hist(bins=np.logspace(start=0, stop=6, num=50))
ax.set_xscale("log")
ax.set_xlabel("Instructions in Parallel region")
ax.set_title("Distribution of Instruction counts of Parallel region")
plt.savefig(os.path.join(plot_path, "weighted_instructions_distribution.pdf"))

In [ ]:
df_overhead_rating = df.groupby("Code")[["large_overhead", "small_overhead"]].sum()
df_overhead_rating['percentage'] = df_overhead_rating['large_overhead'] / (
        df_overhead_rating['large_overhead'] + df_overhead_rating['small_overhead'])

In [ ]:
# plot an excerpt
ax = df_overhead_rating.iloc[0:10].plot.barh(stacked=True)

In [ ]:
# Repos with at least one example of large overhead region
(df_overhead_rating["large_overhead"] > 1).sum()

In [ ]:
ax = df_overhead_rating['percentage'].plot.hist(bins=100)
ax.set_xlabel("Fraction of high overhead parallel regions")
ax.set_ylabel("number of Codes")
ax.set_title("Fraction of high overhead parallel regions per project")

#plt.savefig(os.path.join(plot_path,"plot.pdf"))

In [ ]:
fig, ax = plt.subplots(figsize=(10, 2))

data = df_overhead_rating['percentage'].sort_values()

# Normalize the percentage values to use as colors
# Normalize the percentage values to use as colors
norm = plt.Normalize(data.min(), data.max())
sm = plt.cm.ScalarMappable(cmap="cool", norm=norm)
colors = sm.to_rgba(data)

# Plot the horizontal bar segments
start = 0
for i, percentage in enumerate(data):
    ax.barh(0, 1, left=start, color=colors[i]
            #, edgecolor='black'
            )
    start += 1

# Set the labels and title
ax.set_xlabel("Number of Codes")
ax.set_yticks([])  # Hide y-axis ticks
ax.set_xlim([0, len(data)])
ax.set_title("Fraction of high overhead parallel regions per project")

# Add a legend
cbar = plt.colorbar(sm, ax=ax, orientation='horizontal', pad=0.4)
cbar.set_label('Fraction of high overhead parallel regions')
fig.tight_layout()
plt.savefig(os.path.join(plot_path, "fraction_per_project.pdf"))
plt.show()


In [ ]:
#df_loops = df.groupby("Code")[["default_tripcount_loops","known_tripcount_loops","thread_dependant_trip_count_loops"]].sum()

In [ ]:
df["loop_sum"] = df[["default_tripcount_loops", "known_tripcount_loops", "thread_dependant_trip_count_loops"]].sum(
    axis=1)

In [ ]:
df

In [ ]:
ax = df["loop_sum"].plot.hist(bins=np.logspace(start=0, stop=2, num=20))
ax.set_xscale("log")
ax.set_xlabel("Loops in Parallel region")
ax.set_title("Distribution of loop counts in Parallel region")
#plt.savefig(os.path.join(plot_path, "loops_distribution.pdf"))

In [ ]:
# manual sorting into the bins
bins = np.logspace(start=0, stop=2, num=21)

default = [0 for _ in range(len(bins) + 1)]
known = [0 for _ in range(len(bins) + 1)]
thread = [0 for _ in range(len(bins) + 1)]

for idx, row in df.iterrows():
    if row['loop_sum'] > 0:
        b_index = np.searchsorted(bins, row["loop_sum"])
        default[b_index] += row["default_tripcount_loops"] / row["loop_sum"]
        known[b_index] += row["known_tripcount_loops"] / row["loop_sum"]
        thread[b_index] += row["thread_dependant_trip_count_loops"] / row["loop_sum"]

default = default[:-2]
known = known[:-2]
thread = thread[:-2]

fig, ax = plt.subplots()
ax.bar(
    bins[:-1],
    default,
    label="default_tripcount_loops",
    width=np.diff(bins),
    align='edge',
)
ax.bar(
    bins[:-1],
    known,
    bottom=default,
    label="known_tripcount_loops",
    width=np.diff(bins),
    align='edge',
)
ax.bar(
    bins[:-1],
    thread,
    bottom=np.add(default, known),
    label="thread_dependant_tripcount_loops",
    width=np.diff(bins),
    align='edge',
)

ax.set_xscale("log")
ax.set_xlabel("Loops in Parallel region")
ax.set_ylabel("Frequency")
# bar coloring according to percentages of loops
ax.set_title("Distribution of loop counts in Parallel region")
ax.legend()

plt.savefig(os.path.join(plot_path, "loops_distribution.pdf"))
plt.show()

In [ ]:
# per file statistics
df_parallel_portion = df.groupby(["Code", "File"]).agg(
    instructions_flat=pd.NamedAgg(column="instructions_flat", aggfunc="sum"),
    TOTAL_FILE_INSTR_COUNT=pd.NamedAgg(column="TOTAL_FILE_INSTR_COUNT", aggfunc="max")
).reset_index()
# per project
df_parallel_portion = df_parallel_portion.groupby("Code")[["instructions_flat", "TOTAL_FILE_INSTR_COUNT"]].sum()
df_parallel_portion["percent_parallel"] = df_parallel_portion["instructions_flat"] / (
            df_parallel_portion["instructions_flat"] + df_parallel_portion["TOTAL_FILE_INSTR_COUNT"])
df_parallel_portion

In [ ]:
fig, ax = plt.subplots(figsize=(10, 2))

data = df_parallel_portion['percent_parallel'].sort_values()

# Normalize the percentage values to use as colors
# Normalize the percentage values to use as colors
norm = plt.Normalize(data.min(), data.max())
sm = plt.cm.ScalarMappable(cmap=plt.colormaps.get_cmap("cool").reversed(), norm=norm)
colors = sm.to_rgba(data)

# Plot the horizontal bar segments
start = 0
for i, percentage in enumerate(data):
    ax.barh(0, 1, left=start, color=colors[i]
            #, edgecolor='black'
            )
    start += 1

# Set the labels and title
ax.set_xlabel("Number of Codes")
ax.set_yticks([])  # Hide y-axis ticks
ax.set_xlim([0, len(data)])
ax.set_title("Fraction of assembly instructions inside of Parallel Regions")

# Add a legend
cbar = plt.colorbar(sm, ax=ax, orientation='horizontal', pad=0.4)
cbar.set_label('Fraction of assembly instructions inside of Parallel Regions')
fig.tight_layout()
plt.savefig(os.path.join(plot_path, "fraction_of_parallel_region.pdf"))
plt.show()

In [ ]:
#pd.to_datetime(df_repo_metadata["PushDate"])
#for schedule(dynamic, 10) nowait

In [ ]:
def get_closing_brace(s, start_pos):
    """
    Returns the position of the corresponding closing brace given a string and the position of an opening brace.

    Parameters:
    s (str): The string to search for the closing brace.
    start_pos (int): The position of the opening brace.

    Returns:
    int: The position of the closing brace, or -1 if not found.
    """
    # get the opening brace character
    open_brace = s[start_pos]

    # determine the closing brace character
    if open_brace == '(':
        close_brace = ')'
    elif open_brace == '[':
        close_brace = ']'
    elif open_brace == '{':
        close_brace = '}'
    else:
        # not a valid opening brace character
        return -1

    # search for the closing brace
    count = 1
    pos = start_pos + 1
    while pos < len(s) and count > 0:
        if s[pos] == open_brace:
            count += 1
        elif s[pos] == close_brace:
            count -= 1
        pos += 1

    # return the position of the closing brace
    if count == 0:
        return pos - 1
    else:
        return -1

def tokenize_pragma(pragma):
    # removes extra specification in the braces
    without_braces = pragma
    while '(' in without_braces:
        index = without_braces.find('(')
        closing = get_closing_brace(without_braces, index)
        without_braces = (without_braces[:index] + without_braces[closing+1:])    
    return without_braces.strip().split(' ')

df_pragmas["openmp_pragma_tokenized"] = df_pragmas["openmp_pragma_used"].apply(tokenize_pragma)
all_clauses = set([clause for clauses in df_pragmas["openmp_pragma_tokenized"] for clause in clauses])
all_clauses.remove('')

for c in all_clauses:
    df_pragmas[c] = df_pragmas['openmp_pragma_tokenized'].apply(lambda x: c in x)
df_pragmas["parallel_for"]=df_pragmas["parallel"] & df_pragmas["for"]
# Set 'parallel' and 'for' columns to False where 'parallel_for' is True
df_pragmas.loc[df_pragmas['parallel_for'], ['parallel', 'for']] = False
df_pragmas = df_pragmas.copy()


In [ ]:
#df_pragmas.groupby(["Code"]).size().sort_values(ascending=False)
ax = df_pragmas.groupby("Code").size().value_counts().sort_index().plot.bar()
ax.set_xlabel("Number of OpenMP Pragmas")
ax.set_ylabel("number of Codes")
ax.set_title("Number of OpenMP pragmas per Repo")
plt.savefig(os.path.join(plot_path, "num_pragmas.pdf"))
plt.tight_layout()
plt.show()


In [ ]:
df_pragmas.groupby(["Code"]).sum()




In [ ]:
ax = df_pragmas.groupby(["Code"])["for"].sum().value_counts().sort_index().plot.bar()
ax.set_xlabel("Number of For Pragmas")
ax.set_ylabel("number of Codes")
ax.set_title("Number of omp for pragmas per Repo")
plt.savefig(os.path.join(plot_path, "num_for_pragma.pdf"))
plt.tight_layout()
plt.show()

In [ ]:
ax = df_pragmas.groupby(["Code"])["parallel"].sum().value_counts().sort_index().plot.bar()
ax.set_xlabel("Number of parallel Pragmas")
ax.set_ylabel("number of Codes")
ax.set_title("Number of omp parallel pragmas per Repo")
plt.savefig(os.path.join(plot_path, "num_parallel_pragma.pdf"))
plt.tight_layout()
plt.show()

In [ ]:
ax = df_pragmas.groupby(["Code"])["task"].sum().value_counts().sort_index().plot.bar()
ax.set_xlabel("Number of task Pragmas")
ax.set_ylabel("number of Codes")
ax.set_title("Number of omp task pragmas per Repo")
plt.savefig(os.path.join(plot_path, "num_task_pragma.pdf"))
plt.tight_layout()
plt.show()

In [ ]:
pivot_table = pd.pivot_table(df_pragmas,columns="Code",values=all_clauses.union(["parallel_for"]),aggfunc="sum")
# parallel for is both parallel and for
pivot_table.loc["parallel"] = pivot_table.loc["parallel"] - pivot_table.loc["parallel_for"] 
pivot_table.loc["for"] = pivot_table.loc["for"] - pivot_table.loc["parallel_for"]

In [ ]:
pivot_table.sum(axis=1).sort_values(ascending=False)

In [ ]:
#to_view = ["parallel","for","parallel_for","task","section","critical","single","simd","atomic","barrier"]
to_view = ["parallel","for","parallel_for","task","section","critical","single","atomic","barrier"]
not_to_view=  ["Code","src_location", "src_location_line_number","call","openmp_pragma_used","openmp_pragma_tokenized"]
relevant_cols = [c for c in df_pragmas.columns if c not in not_to_view]

In [ ]:
grouping = df_pragmas.groupby(["Code"]).sum()
# manual sorting into the bins
bins = [1,2,3,4,5,10,20,50,100,500,1000]

cat_data = {key : [0 for _ in range(len(bins) + 1)] for key in to_view+ ["other"]}

for idx, row in grouping.iterrows():
    sum_pragmas = df_pragmas.groupby("Code").size()[row.name]
    if  sum_pragmas > 0:
        b_index = np.searchsorted(bins, sum_pragmas)
        sum_shown = 0
        for cat in to_view:
            cat_data[cat][b_index] += row[cat] /sum_pragmas
            sum_shown += row[cat]
        cat_data["other"][b_index] += (sum_pragmas - sum_shown) /sum_pragmas
            
   
for cat in to_view+ ["other"]:
    cat_data[cat] = cat_data[cat][:-2]


fig, ax = plt.subplots()
bot = np.zeros(len(bins)-1)
for cat in to_view+ ["other"]:    
    ax.bar(
        [1,2,3,4,5,6,7,8,9,10],
        cat_data[cat],
        label=cat,
        #width=10,
        align='edge',
        bottom=bot
    )
    bot += cat_data[cat]

#ax.set_xscale("log")
ax.set_xlabel("OpenMP Pragmas per repository")
ax.set_xticks(ticks= np.array([1,2,3,4,5,6,7,8,9,10])+ 0.4,labels=[1,2,3,4,5,"<10","<20","<50","<100","<500"])
ax.set_ylabel("Frequency")
# bar coloring according to percentages of loops
ax.set_title("Number of OpenMP pragmas per repository")
ax.legend()

plt.savefig(os.path.join(plot_path, "num_pragmas.pdf"))
plt.show()